In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from collections import Counter
%matplotlib inline

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/airplane-crashes-since-1908/Airplane_Crashes_and_Fatalities_Since_1908.csv


# Reading in the dataset

In [2]:
df = pd.read_csv("/kaggle/input/airplane-crashes-since-1908/Airplane_Crashes_and_Fatalities_Since_1908.csv")
df.head()

,Date,Time,Location,Operator,Flight #,Route,Type,Registration,cn/In,Aboard,Fatalities,Ground,Summary
0,09/17/1908,17:18,"Fort Myer, Virginia",Military - U.S. Army,NaN,Demonstration,Wright Flyer III,NaN,1,2.0,1.0,0.0,"During a demonstration flight, a U.S. Army fly..."
1,07/12/1912,06:30,"AtlantiCity, New Jersey",Military - U.S. Navy,NaN,Test flight,Dirigible,NaN,NaN,5.0,5.0,0.0,First U.S. dirigible Akron exploded just offsh...
2,08/06/1913,NaN,"Victoria, British Columbia, Canada",Private,-,NaN,Curtiss seaplane,NaN,NaN,1.0,1.0,0.0,The first fatal airplane accident in Canada oc...
3,09/09/1913,18:30,Over the North Sea,Military - German Navy,NaN,NaN,Zeppelin L-1 (airship),NaN,NaN,20.0,14.0,0.0,The airship flew into a thunderstorm and encou...
4,10/17/1913,10:30,"Near Johannisthal, Germany",Military - German Navy,NaN,NaN,Zeppelin L-2 (airship),NaN,NaN,30.0,30.0,0.0,Hydrogen gas which was being vented was sucked...


# Objective:
I´d like to analyze and predict airplane crashes and the likelihood of surviving such a crash (although I couldn´t find any data on passenger seating so far). Goal is to answer the question wether flying really is the safest way to travel.

# 1. data cleaning:
# 1.1. remove columns with 80 ore more percent of Null-values

In [3]:
df.info() #5268 rows
5268*0.8 #4214 rows needed at least
5268 - 4214 #1054 <- breakpoint for non-null cols to delete:
# cols to delete:
# Flight # --> 1069 non-null cols (close call!)
# no cols to delete!

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5268 entries, 0 to 5267
Data columns (total 13 columns):
Date            5268 non-null object
Time            3049 non-null object
Location        5248 non-null object
Operator        5250 non-null object
Flight #        1069 non-null object
Route           3562 non-null object
Type            5241 non-null object
Registration    4933 non-null object
cn/In           4040 non-null object
Aboard          5246 non-null float64
Fatalities      5256 non-null float64
Ground          5246 non-null float64
Summary         4878 non-null object
dtypes: float64(3), object(10)
memory usage: 535.2+ KB


1054

No columns to delte, although `Flight #` was a close call

# 1.2: check the `Ground` column to see if it only contains `0` and `1`. If so convert it to a Boolean type

In [4]:
len(df.Ground.unique())

51

## there are 51 unique values in the `Ground` column. Unfortunately the dataset doesn´t provide information about this column and it´s values

# questions to answer:
# 2.1: on avg. how many plane crashes happen per year?

In [5]:
#converting the Date column to datetime Object:

df["Date"] = pd.to_datetime(df.Date).copy()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5268 entries, 0 to 5267
Data columns (total 13 columns):
Date            5268 non-null datetime64[ns]
Time            3049 non-null object
Location        5248 non-null object
Operator        5250 non-null object
Flight #        1069 non-null object
Route           3562 non-null object
Type            5241 non-null object
Registration    4933 non-null object
cn/In           4040 non-null object
Aboard          5246 non-null float64
Fatalities      5256 non-null float64
Ground          5246 non-null float64
Summary         4878 non-null object
dtypes: datetime64[ns](1), float64(3), object(9)
memory usage: 535.2+ KB


In [6]:
df.Date.dt.year.value_counts() # 98 years were documented
yearly_avg_crashes = len(df.Date)/98
yearly_avg_crashes

53.755102040816325

## on avg. there occured 53 crashes each year. However we need to take into consideration, that as time progressed the number of flights also increased! Thus the Average is pretty skewed.

## 2.1.1: which year was the one with the least/most crashes?

In [7]:
df.Date.dt.year.value_counts(ascending=False, sort=True)

1972    104
1968     96
1989     95
1967     91
1973     89
       ... 
1918      4
1913      3
1915      2
1912      1
1908      1
Name: Date, Length: 98, dtype: int64

## 1908 and 1912 were the years with the least (only 1) crashes --> to no surprise, since aviation was still in it´s infancy then
## 1972 was the year with the most crashes (104)

# 2.2: on avg. how many months are in between crashes?

In [8]:
# sort dataframe by Date:
df.sort_values(by=['Date'], ascending=True)

# create new column for the timedelta:
df["Time between crashes"] = df.Date.diff()
df.head()

# calculate the average of the new column:
df["Time between crashes"].mean()

Timedelta('6 days 23:38:07.677995')

## On average a plane crashes every 6-7 days

## 2.2.1: what was the longest period in between crashes?

In [9]:
# sort dataframe by Date:
df.sort_values(by=['Date'], ascending=True)
df["Time between crashes"].max() # max crash-free days was 1394
# convert 1394 days to years:
years_between_crashes = 1394/365
years_between_crashes

3.819178082191781

## the longest period in between crashes was 1394 days or almost 4 years!

In [10]:
df[df["Time between crashes"].dt.days==1394] # index 1
df[:2]

,Date,Time,Location,Operator,Flight #,Route,Type,Registration,cn/In,Aboard,Fatalities,Ground,Summary,Time between crashes
0,1908-09-17,17:18,"Fort Myer, Virginia",Military - U.S. Army,NaN,Demonstration,Wright Flyer III,NaN,1,2.0,1.0,0.0,"During a demonstration flight, a U.S. Army fly...",NaT
1,1912-07-12,06:30,"AtlantiCity, New Jersey",Military - U.S. Navy,NaN,Test flight,Dirigible,NaN,NaN,5.0,5.0,0.0,First U.S. dirigible Akron exploded just offsh...,1394 days


# the longest period between crashes was 1908 - 1912

# 2.3: which Operator suffered the most crashes in history?

In [11]:
df.Operator.value_counts(sort=True, ascending=False)

Aeroflot                                                  179
Military - U.S. Air Force                                 176
Air France                                                 70
Deutsche Lufthansa                                         65
China National Aviation Corporation                        44
                                                         ... 
Aero Eslava                                                 1
Winship Air Service                                         1
Military - U.S. Air Force / Military -  U.S. Air Force      1
MerriAviation - Air Taxi                                    1
Blue Wing Airlines                                          1
Name: Operator, Length: 2476, dtype: int64

## Aeroflot (179 crashes) and the U.S. Air Force (176 crashes) suffered the most crashes in history. Being Military operators that´s no surprise. The civil operator with the most crashes is Air France

# 3.1: from the Route column exract start and destination and create new columns for each

In [12]:
df.Route.isnull().sum() # 1706 Null Values
df = df.dropna(subset=["Route"])
df.Route.isnull().sum()

0

In [13]:
divider = df.Route.str.contains("-")
df.Route[divider] # 3362 rows that use "-" as divider
df.Route[~divider] # 200 rows that don´t use above pattern 
df.Route.isnull().sum()

0

In [14]:
#create an empty list to store the separated column values in:
route = []
route.append(df.Route.str.split("-",expand=True))
    
# create a new dataframe:
route = route[0]
print(route.shape)
print(df.shape)
#as both dataframes have the same no. of rows we can safely merge them:
flights = pd.concat([df, route], axis=1)
# rename the new columns:
flights.rename(columns={0:"Start",1:"Destination"},inplace=True)
flights.head()

(3562, 6)
(3562, 14)


,Date,Time,Location,Operator,Flight #,Route,Type,Registration,cn/In,Aboard,Fatalities,Ground,Summary,Time between crashes,Start,Destination,2,3,4,5
0,1908-09-17,17:18,"Fort Myer, Virginia",Military - U.S. Army,NaN,Demonstration,Wright Flyer III,NaN,1,2.0,1.0,0.0,"During a demonstration flight, a U.S. Army fly...",NaT,Demonstration,None,None,None,None,None
1,1912-07-12,06:30,"AtlantiCity, New Jersey",Military - U.S. Navy,NaN,Test flight,Dirigible,NaN,NaN,5.0,5.0,0.0,First U.S. dirigible Akron exploded just offsh...,1394 days,Test flight,None,None,None,None,None
56,1921-09-06,NaN,"Paris, France",Franco-Roumaine,NaN,Varsovie - Strasbourg - Paris,Potez IX,F-ADCD,160,5.0,5.0,0.0,Crashed while making an approach to Le Bourget...,13 days,Varsovie,Strasbourg,Paris,None,None,None
80,1923-12-23,02:30,Over the Mediterranean Sea,Military - French Navy,NaN,Toulon - Algiers,Zeppelin Dixmunde (airship),L-72,NaN,52.0,52.0,0.0,"Crashed while on a flight from Toulon, France ...",17 days,Toulon,Algiers,None,None,None,None
83,1924-04-24,NaN,Over the English Channel,KLM Royal Dutch Airlines,NaN,"Lympne, England - Rotterdam, The Netherlands",Fokker F.III,H-NABS,1535,3.0,3.0,0.0,NaN,48 days,"Lympne, England","Rotterdam, The Netherlands",None,None,None,None


In [15]:
# move Values (if there are any) from columns 2 - 5 to the Destination col:
flights.Destination.fillna(value=flights[2],inplace=True)
flights.Destination.isnull().sum() 

flights.Destination.fillna(value=flights[3],inplace=True)
flights.Destination.isnull().sum() 

flights.Destination.fillna(value=flights[4],inplace=True)
flights.Destination.isnull().sum() 

flights.Destination.fillna(value=flights[5],inplace=True)
flights.Destination.isnull().sum() 

flights[["Start","Destination"]].head()

,Start,Destination
0,Demonstration,None
1,Test flight,None
56,Varsovie,Strasbourg
80,Toulon,Algiers
83,"Lympne, England","Rotterdam, The Netherlands"


In [16]:
# checking left-over null values:
flights.Destination.isnull().sum()

200

## these are the 200 rows we earlier identified as not using `-` as delimiter, thus not having a "from - to" route

In [17]:
flights[flights.Destination.isnull()].head()

,Date,Time,Location,Operator,Flight #,Route,Type,Registration,cn/In,Aboard,Fatalities,Ground,Summary,Time between crashes,Start,Destination,2,3,4,5
0,1908-09-17,17:18,"Fort Myer, Virginia",Military - U.S. Army,NaN,Demonstration,Wright Flyer III,NaN,1,2.0,1.0,0.0,"During a demonstration flight, a U.S. Army fly...",NaT,Demonstration,None,None,None,None,None
1,1912-07-12,06:30,"AtlantiCity, New Jersey",Military - U.S. Navy,NaN,Test flight,Dirigible,NaN,NaN,5.0,5.0,0.0,First U.S. dirigible Akron exploded just offsh...,1394 days,Test flight,None,None,None,None,None
121,1927-09-17,NaN,"Hadley, New Jersey",Reynolds Airways,NaN,Sightseeing,Fokker F-VII,NC776,4840,12.0,7.0,0.0,The sightseeing plane took off and rose to 400...,14 days,Sightseeing,None,None,None,None,None
148,1928-07-13,NaN,"Purley, England",Imperial Airways,NaN,Testing,Vickers 74 Vulcan,G-EBLB,9,4.0,4.0,0.0,Crashed to the ground shortly after taking off...,3 days,Testing,None,None,None,None,None
174,1929-03-17,NaN,"Newark, New Jersey",Colonial Western Airlines,NaN,Sightseeing,Ford 5-AT-B Tri Motor,NC7683,4-AT-41,15.0,14.0,0.0,The plane was making the last of a number of s...,45 days,Sightseeing,None,None,None,None,None


### we can now drop columns 2,3 and 4

In [18]:
flights.drop(columns=[2,3,4,5], inplace=True)

In [19]:
flights.head()

,Date,Time,Location,Operator,Flight #,Route,Type,Registration,cn/In,Aboard,Fatalities,Ground,Summary,Time between crashes,Start,Destination
0,1908-09-17,17:18,"Fort Myer, Virginia",Military - U.S. Army,NaN,Demonstration,Wright Flyer III,NaN,1,2.0,1.0,0.0,"During a demonstration flight, a U.S. Army fly...",NaT,Demonstration,None
1,1912-07-12,06:30,"AtlantiCity, New Jersey",Military - U.S. Navy,NaN,Test flight,Dirigible,NaN,NaN,5.0,5.0,0.0,First U.S. dirigible Akron exploded just offsh...,1394 days,Test flight,None
56,1921-09-06,NaN,"Paris, France",Franco-Roumaine,NaN,Varsovie - Strasbourg - Paris,Potez IX,F-ADCD,160,5.0,5.0,0.0,Crashed while making an approach to Le Bourget...,13 days,Varsovie,Strasbourg
80,1923-12-23,02:30,Over the Mediterranean Sea,Military - French Navy,NaN,Toulon - Algiers,Zeppelin Dixmunde (airship),L-72,NaN,52.0,52.0,0.0,"Crashed while on a flight from Toulon, France ...",17 days,Toulon,Algiers
83,1924-04-24,NaN,Over the English Channel,KLM Royal Dutch Airlines,NaN,"Lympne, England - Rotterdam, The Netherlands",Fokker F.III,H-NABS,1535,3.0,3.0,0.0,NaN,48 days,"Lympne, England","Rotterdam, The Netherlands"


# 3.2: what´s the most common weekday for crashes?

In [20]:
flights.Date.dt.weekday.value_counts(sort=True, ascending=False)

3    551
4    551
2    551
1    515
5    509
0    478
6    407
Name: Date, dtype: int64

## most crashes happen on Wednesdays,Thursdays and Fridays

# 4.1: on avg. how many passengers survive a crash (if any)? Create a new column called `survival rate`

In [21]:
flights[["Aboard", "Fatalities"]].describe()

,Aboard,Fatalities
count,3557.000000,3560.000000
mean,33.093337,23.529494
std,49.516421,38.046322
min,1.000000,0.000000
25%,6.000000,3.000000
50%,16.000000,10.000000
75%,38.000000,26.000000
max,644.000000,583.000000


In [22]:
flights["survival rate"] = 1-(flights.Fatalities/flights.Aboard)

In [23]:
flights["survival rate"].mean()*100

17.791031958517628

## the average survival rate is 17.79%

# Bonus
## - What were the most common causes for crashes? Could be extraced from the Summarycolumn (e.g. most common words)

In [24]:
# split all the paragraphs of a column into a list of words:
#word_list = flights.Summary.str.split()

# the list contains integers, we need to remove them or else the counter will throw an error:
#no_integers = [x for x in word_list if not isinstance(x, float)]

# flatten the list of lists:
#flat_list = []
#for sublist in no_integers:
#    str(sublist)
#    flat_list.append(sublist)     

In [25]:
# Pass the flat_it list to instance of Counter class. 
#count_dict = {}
#for item in flat_list:
 #   zahler = Counter(item)
  #  count_dict[zahler] = item
# most_common() produces k frequently encountered 
# input values and their respective counts. 
#most_occur = count_dict.most_common(100) 
  
#len(most_occur)